In [1]:
import requests

def get_estimated_timetable(stop_point_ref):
    url = "https://prim.iledefrance-mobilites.fr/marketplace/stop-monitoring"

    params = {
        "MonitoringRef": stop_point_ref
    }

    headers = {
        "apikey": "VKl2FLcQCLZXZxJfP65faKkRRxSHWdHX"
    }

    # Effectuer la requête GET
    response = requests.get(url, params=params, headers=headers)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        data = response.json()  
        return data
    else:
        print("Failed to fetch data: Status code", response.status_code)
        return None

In [40]:
stopRef = "STIF:StopPoint:Q:22061:"
data = get_estimated_timetable(stopRef)
destination_display = data['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['DestinationDisplay'][0]['value']
expected_arrival_time = data['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['ExpectedArrivalTime']

res = dict({stopRef: {"destination": destination_display,"expected_arrival_time":expected_arrival_time }})

In [54]:
print(res)

{'STIF:StopPoint:Q:22061:': {'destination': "Mairie d'Aubervilliers", 'expected_arrival_time': '2024-06-15T09:05:05.028Z'}}


In [56]:
from kafka import KafkaProducer
import json

#localhost:9092
#kafka1:19092

def send(res):
    try:
        producer = KafkaProducer(
            bootstrap_servers=['localhost:9092'],
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda v: v.encode('utf-8')
        )

        # Loop through the res dictionary and send each key-value pair
        for key, value in res.items():
            producer.send('infosRealTime', key=key, value=value)

        producer.flush()  # Ensure all messages are sent before closing
        producer.close()
        print("Message sent successfully.")
    except Exception as e:
        print(f"Error sending message: {e}")


In [57]:
send(res)

Message sent successfully.


In [ ]:
try:
        producer = KafkaProducer(
            bootstrap_servers=['localhost:9092'],
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda v: v.encode('utf-8')
        )

        # Loop through the res dictionary and send each key-value pair
        for key, value in res.items():
            producer.send('infosRealTime', key=key, value=value)

        producer.flush()  # Ensure all messages are sent before closing
        producer.close()
        print("Message sent successfully.")
    except Exception as e:
        print(f"Error sending message: {e}")